## Chapter 5:  Exercise 5.1

### 5.1

Show that the truncated power basis function in (5.3) represent a basis for a cubic spline with the two knots as indicated.

$$
\begin{align}
h_0(X) = 1, \quad & h_2(X) = X^2 \quad h_4(X) = \left( X - \zeta_0 \right)^3_{+} \\
h_1(X) = X, \quad & h_3(X) = X^3 \quad h_5(X) = \left( X - \zeta_1 \right)^3_{+} \\
\end{align}
$$


#### Solution:

The problem is definen for three regions: $(-\infty, \zeta_1); [\zeta_1, \zeta_2); [\zeta_2, \infty)$, two knots at $\zeta_1$ and $\zeta_2$, we are using an order-3 spline. Then

$$
f(X) = 
\begin{cases}
\beta_0 + \beta_1 X + \beta_2 X^2 + \beta_3 X^3 & X \le \zeta_0 \\
\beta_4 + \beta_5 X + \beta_6 X^2 + \beta_7 X^3 & \zeta_0 \leq X \le \zeta_1 \\
\beta_8 + \beta_{9} X + \beta_{10} X^2 + \beta_{11} X^3 & \zeta_1 \leq X
\end{cases}
$$

We have three restrictions per of continuity knot (one for each derivative w.r.t $X$):

$$
\begin{align}
\beta_0 + \beta_1 \zeta_0 + \beta_2 \zeta_0^2 + \beta_3 \zeta_0^3 & =  \beta_4 + \beta_5 \zeta_0 + \beta_6 \zeta_0^2 + \beta_7 \zeta_0^3 \\
\beta_1 + 2 \beta_2 \zeta_0 + 3 \beta_3 \zeta_0^2 & =  \beta_5 + 2 \beta_6 \zeta_0 + 3 \beta_7 \zeta_0^2 \\
2 \beta_2 + 6 \beta_3 \zeta_0 & =  2 \beta_6 + 6 \beta_7 \zeta_1 \\
\end{align}
$$

And for $\zeta_1$

$$
\begin{align}
\beta_4 + \beta_5 \zeta_1 + \beta_6 \zeta_1^2 + \beta_7 \zeta_1^3 & =  \beta_8 + \beta_{9} \zeta_1 + \beta_{10} \zeta_1^2 + \beta_{11} \zeta_1^3 \\
\beta_5 + 2 \beta_6 \zeta_1 + 3 \beta_7 \zeta_1^2 & =  \beta_{9} + 2 \beta_{10} \zeta_1 + 3 \beta_{11} \zeta_1^2 \\
2 \beta_6 + 6 \beta_7 \zeta_1 & =  2 \beta_{10} + 6 \beta_{11} \zeta_1 \\
\end{align}
$$


This is a six-dimensional linear space: (3 regions) $\times$ (4 parameters per region) - (2 knots) $\times$ (3 restritictions per knot) = 6 parameters.

The first step is to express $f(X)$ in terms of 6 parameters $\beta_1$ by reeplacing some of them according to the constraints.

In [3]:
import sympy as sp

b = sp.Matrix([sp.Symbol(f'beta_{i}') for i in range(12)])
z = sp.Matrix([sp.Symbol(f'zeta_{i}') for i in range(2)])
x = sp.Matrix([sp.Symbol('X')**i for i in range(4)])

f1 = b[0] + b[1]*x[1] + b[2]*x[2] + b[3]*x[3]
f2 = b[4] + b[5]*x[1] + b[6]*x[2] + b[7]*x[3]
f3 = b[8] + b[9]*x[1] + b[10]*x[2] + b[11]*x[3]

f = sp.Piecewise(
    (f1, x[1] < z[0]),
    (f2, ((z[0] <= x[1]) & (x[1] < z[1]))),
    (f3, z[1] <= x[1])
)

# print('f(x) = ')
f

Piecewise((X**3*beta_3 + X**2*beta_2 + X*beta_1 + beta_0, X < zeta_0), (X**3*beta_7 + X**2*beta_6 + X*beta_5 + beta_4, X < zeta_1), (X**3*beta_11 + X**2*beta_10 + X*beta_9 + beta_8, True))

### Solving 6 parameters
The following code replaces parameters $\beta_0, \beta_1, \beta_2, \beta_8, \beta_9, \beta_10$ that for convience can be expressed in term of the $\beta_5, \beta_6, \beta_7$

In [4]:
def solver(f_a, f_b, zeta, beta, diff_order):
    solved_beta = sp.solve((
        sp.diff(f_a, x[1], diff_order).subs(x[1], zeta) - 
        sp.diff(f_b, x[1], diff_order).subs(x[1], zeta)
        ), beta
    )[0]
    
    return solved_beta

solved_b0 = solver(f1, f2, z[0], b[0], 0)
solved_b1 = solver(f1, f2, z[0], b[1], 1)
solved_b2 = solver(f1, f2, z[0], b[2], 2)

solved_b8 = solver(f2, f3, z[1], b[8], 0)
solved_b9 = solver(f2, f3, z[1], b[9], 1)
solved_b10 = solver(f2, f3, z[1], b[10], 2)

f1_solved = sp.factor(sp.simplify(f1
    .subs(b[0], solved_b0)
    .subs(b[1], solved_b1)
    .subs(b[2], solved_b2)
), x[1])

f3_solved = sp.factor(sp.simplify(f3
    .subs(b[8], solved_b8)
    .subs(b[9], solved_b9)
    .subs(b[10], solved_b10)
), x[1])

f = sp.Piecewise(
    (f1_solved, x[1] < z[0]),
    (f2, ((z[0] <= x[1]) & (x[1] < z[1]))),
    (f3_solved, z[1] <= x[1])
)

Ending up in:

$$
f(X) = \begin{cases}
X^{3} \beta_{3} + X^{2} \left(- 3 \beta_{3} \zeta_{0} + \beta_{6} + 3 \beta_{7} \zeta_{0}\right) + X \left(3 \beta_{3} \zeta_{0}^{2} + \beta_{5} - 3 \beta_{7} \zeta_{0}^{2}\right) - \beta_{3} \zeta_{0}^{3} + \beta_{4} + \beta_{7} \zeta_{0}^{3} & \text{for}\: X < \zeta_{0} \\
X^{3} \beta_{7} + X^{2} \beta_{6} + X \beta_{5} + \beta_{4} & \text{for}\: \zeta_{0} <= X < \zeta_{1} \\
X^{3} \beta_{11} + X^{2} \left(- 3 \beta_{11} \zeta_{1} + \beta_{6} + 3 \beta_{7} \zeta_{1}\right) + X \left(3 \beta_{11} \zeta_{1}^{2} + \beta_{5} - 3 \beta_{7} \zeta_{1}^{2}\right) - \beta_{11} \zeta_{1}^{3} + \beta_{4} + \beta_{7} \zeta_{1}^{3} & \zeta_{1} <= X
\end{cases}
$$

We can re-write the  $f(X)$ using picewise linear basis functions that change the output according to the regions.

$$
f(X) = f_1(X) + f_2(X) I(\zeta_0 <= X) + f_3(X)I(\zeta_1 <= X)
$$

Where $I(True) = 1$ and $I(False) = 0$. Based on the piecewise definition of $f(X)$ we have that:

$$
\begin{align}
f_1(X) & = \beta_{3} + X^{2} \left(- 3 \beta_{3} \zeta_{0} + \beta_{6} + 3 \beta_{7} \zeta_{0}\right) + X \left(3 \beta_{3} \zeta_{0}^{2} + \beta_{5} - 3 \beta_{7} \zeta_{0}^{2}\right) - \beta_{3} \zeta_{0}^{3} + \beta_{4} + \beta_{7} \zeta_{0}^{3} \\
\end{align}
$$

We could define $f_2(X)$ and $f_3(X)$ are order 3 polynomial such as first and second order derivatives are continous, at there is continuity at the knots

$$
\begin{align}
f_2(X) & = \alpha_4 (X - \zeta_0)^3_{+} = \begin{cases}
\alpha_4 (X - \zeta_0)^3_{+} & X > \zeta_0 \\
0 & \text{otherwise}
\end{cases} \\
f_3(X) & = \alpha_5 (X - \zeta_1)^3_{+} = \begin{cases}
\alpha_5 (X - \zeta_1)^3_{+} & X > \zeta_1 \\
0 & \text{otherwise}
\end{cases} \\
f(X) & = \alpha_0 + \alpha_1 X^{2} + \alpha_2 X - \alpha_3 X^{3}  + \alpha_4(X - \zeta_0)^{3}_{+} + \alpha_5(X - \zeta_1)^{3}_{+}
\end{align}
$$

Proving continuous first and second order, and continuity by taking limits at $\zeta_0^-$, $\zeta_0^+$, $\zeta_0^1$, and $\zeta_1^+$ is very trivial. Finally there is a direct equivalence between $\alpha_i$ and $\theta_i$:

$$
\begin{align}
\alpha_0 &= - \beta_{3} \zeta_{0}^{3} + \beta_{4} + \beta_{7} \zeta_{0}^{3} \\
\alpha_1 &= 3 \beta_{3} \zeta_{0}^{2} + \beta_{5} - 3 \beta_{7} \zeta_{0}^{2} \\
\alpha_2 &= - 3 \beta_{3} \zeta_{0} + \beta_{6} + 3 \beta_{7} \zeta_{0}\\
\alpha_3 &= \beta_3
\end{align}
$$


As for $\alpha_4$ and $\alpha_5$ they can be easily derived by saying that for $(\zeta_0 <= X < \zeta_1)$, $f_1(X) + f_2(X) = X^{3} \beta_{7} + X^{2} \beta_{6} + X \beta_{5} + \beta_{4} $ which is the second equation in the piecewise definition of $f(X)$. The same procedure is applied to $f_3(X)$

$$
\begin{align}
\alpha_4 &= \beta_7 - \beta_3 \\
\alpha_5 &= \beta_{11} - \beta_7
\end{align}
$$
